# Типы сервисов

- Пример предназначен для демонстрации работы по получению списка типов сервисов для выбранного `scenario_id`. 
- Особенность заключается в маппинге или адаптации типов сервисов из БД к типам сервисов `blocksnet`.

Список сервисов **уникален** для каждого `project_id`.

In [1]:
SCENARIO_ID = 835

In [2]:
from prostor.fetchers import scenarios

PROJECT_ID = scenarios.get_scenario(SCENARIO_ID)['project']['project_id']
PROJECT_ID

120

## 1. Получение списка сервисов

### 1.2. Получение списка нормативов

Надо как-то получить `normatives` как с эндпоинтом [/api/v1/territory/{territory_id}/normatives](http://10.32.1.65:5300/api/docs#/territories/get_territory_normatives_api_v1_territory__territory_id__normatives_get).

В данном примере цепляю по сути для `territory` из [/api/v1/projects/{project_id}](http://10.32.1.65:5300/api/docs#/projects/get_project_by_id_api_v1_projects__project_id__get). Может стоит спускаться пониже.

In [3]:
from prostor.fetchers import projects

TERRITORY_ID = projects.get_project(PROJECT_ID)['territory']['id']
TERRITORY_ID

1

In [4]:
from prostor.fetchers import territories

normatives_df = territories.get_territory_normatives(TERRITORY_ID)[[
    'radius_availability_meters',
    'time_availability_minutes',
    'services_per_1000_normative',
    'services_capacity_per_1000_normative'
]].copy()
normatives_df.head()

,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative
service_type_id,,,,
28,None,10,NaN,13.0
50,None,60,NaN,3.0
61,None,15,NaN,25.0
105,None,15,NaN,10.0
94,None,30,8.0,NaN


### 1.2. Получение списка сервисов

In [5]:
import prostor.fetchers.misc as misc

service_types_df = misc.get_service_types()
service_types_df['weight'] = service_types_df['properties'].apply(lambda p : p['weight_value'] if 'weight_value' in p else None)
service_types_df = service_types_df[[
    'capacity_modeled',
    'infrastructure_type',
    'weight',
]].copy()
service_types_df.head()

,capacity_modeled,infrastructure_type,weight
service_type_id,,,
1,2000.0,basic,0.2
2,NaN,None,NaN
3,NaN,None,NaN
4,NaN,None,NaN
5,NaN,comfort,0.5


### 1.3. Дружба нормативов и списка сервисов.

Всё собираем в один большой список, чтобы дальше с ним работать.

In [6]:
service_types_df = service_types_df.join(normatives_df)

In [7]:
service_types_df.head()

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative
service_type_id,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0
2,NaN,None,NaN,NaN,NaN,NaN,NaN
3,NaN,None,NaN,NaN,NaN,NaN,NaN
4,NaN,None,NaN,NaN,NaN,NaN,NaN
5,NaN,comfort,0.5,None,60.0,NaN,64.0


### 1.4. Получение списка ценностей

По возможности цепляем каждому service_type_id его список ценностей (`soc_value_id`)

In [8]:
def _get_social_values_ids(service_type_id : int) -> list[int]:
    try:
        social_values_df = misc.get_service_type_social_values(service_type_id)
        return list(social_values_df.index)
    except:
        return None
    
service_types_df['soc_values_ids'] = service_types_df.apply(lambda s : _get_social_values_ids(s.name), axis=1)

In [9]:
service_types_df.head()

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative,soc_values_ids
service_type_id,,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0,"[1, 8, 18]"
2,NaN,None,NaN,NaN,NaN,NaN,NaN,None
3,NaN,None,NaN,NaN,NaN,NaN,NaN,None
4,NaN,None,NaN,NaN,NaN,NaN,NaN,[18]
5,NaN,comfort,0.5,None,60.0,NaN,64.0,None


## 2. Адаптация к блокснет

По возможности даем списку сервисов нейминги из блокснета, чтобы смочь сдружить с оптимизатором и прочим. 

Обычная обеспеченность (`competitive_provision`) отработает и без этого, но ей в Ф36 нужен полный список всё же.

In [10]:
SERVICE_TYPES_MAPPING = {
    1: 'park',
    5 : 'beach',
    21: 'kindergarten',
    22: 'school',
    23 : None, # доп образование
    24 : None, # доп образование
    26 : 'college',
    27 : 'university',
    28 : 'polyclinic',
    29 : None, # детская поликлиника
    30 : None, # стоматология
    31 : None, # фельдшерско-акушерский пункт
    32 : None, # женская консультация
    34 : 'pharmacy',
    35 : 'hospital',
    36 : None, # роддом
    37 : None, # детская больница
    38 : None, # хоспис
    39 : None, # скорая помощь
    40 : None, # травматология
    41 : None, # морг
    42 : None, # диспансер
    43 : None, # центры соц обслуживания
    44 : 'social_facility', # дом престарелых
    45 : 'recruitment',
    46 : None, # детский дом
    47 : 'multifunctional_center',
    48 : 'library',
    49 : None, # дворцы культуры
    50 : 'museum',
    51 : 'theatre',
    53 : None, # концертный зал
    55 : 'zoo',
    56 : 'cinema',
    57 : 'mall', 
    59 : 'stadium',
    60 : None, # ледовая арена
    61 : 'cafe',
    62 : 'restaurant',
    63 : 'bar',
    64 : 'cafe',
    65 : 'bakery',
    66 : 'pitch',
    67 : 'swimming_pool',
    68 : None, # спортивный зал
    69 : None, # каток
    70 : None, # футбольное поле
    72 : None, # эко тропа
    74 : 'playground',
    75 : None, # парк аттракционов
    77 : None, # скейт парк
    78 : 'police',
    79 : None, # пожарная станция
    80 : 'train_station',
    81 : 'train_building',
    82 : 'aeroway_terminal',
    84 : 'fuel',
    86 : 'bus_station',
    88 : 'subway_entrance',
    89 : 'supermarket',
    91 : 'market',
    93 : None, # одежда и обувь
    94 : None, # бытовая техника
    95 : None, # книжный магазин
    96 : None, # детские товары
    97 : None, # спортивный магазин
    98 : 'post',
    99 : None, # пункт выдачи
    100 : 'bank',
    102 : 'lawyer',
    103 : 'notary',
    107 : 'veterinary',
    108 : None, # зоомагазин
    109 : 'dog_park',
    110 : 'hotel',
    111 : 'hostel',
    112 : None, # база отдыха
    113 : None, # памятник
    114 : 'religion', # религиозный объект
    # электростанции -- start
    118 : 'substation', # Атомная электростанция
    119 : 'substation', # Гидро-электростанция
    120 : 'substation', # Тепловая электростанция
    # электростанции -- end
    124 : 'water_works',
    # водоочистные сооружения -- start
    126 : 'wastewater_plant', # Сооружения для очистки воды
    128 : 'wastewater_plant', # Водоочистные сооружения
    # водоочистные сооружения -- end
    143 : 'sanatorium',
}

In [11]:
service_types_df['blocksnet'] = service_types_df.apply(lambda s : SERVICE_TYPES_MAPPING.get(s.name), axis=1)
service_types_df.head()

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative,soc_values_ids,blocksnet
service_type_id,,,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0,"[1, 8, 18]",park
2,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
3,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
4,NaN,None,NaN,NaN,NaN,NaN,NaN,[18],None
5,NaN,comfort,0.5,None,60.0,NaN,64.0,None,beach


## 3. Сохранение списка

In [12]:
service_types_df

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative,soc_values_ids,blocksnet
service_type_id,,,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0,"[1, 8, 18]",park
2,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
3,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
4,NaN,None,NaN,NaN,NaN,NaN,NaN,[18],None
5,NaN,comfort,0.5,None,60.0,NaN,64.0,None,beach
...,...,...,...,...,...,...,...,...,...
133,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
134,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
135,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None


In [13]:
service_types_df.to_pickle('./data/service_types.pickle')